# Using EcoFOCIpy to process raw field data

## Cruise ID - DY2103


## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

In [11]:
import yaml
import glob
import pandas as pd

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

In [12]:
sample_data_dir = '/Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [13]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
nutdatafile = sample_data_dir+'working/DiscreteNutrients/SKQ202115S Nutrient Data.txt' #<- point to cruise and process all files within
cruise_name = 'SKQ202115S' #no hyphens
cruise_meta_file = sample_data_dir+'logs/SKQ202115S.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpNuts.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2021/CTDcasts/SKQ202115S/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2021/CTDc

## Load csv Nutrient File

In [14]:
nut_data = pd.read_csv(nutdatafile,delimiter='\t')
nut_data

,Cast,Niskin,PO4 (uM),Sil (uM),NO3 (uM),NO2 (uM),NH4 (uM)
0,1,3,2.640,88.1,15.8,0.83,12.38
1,1,6,2.615,86.1,15.7,0.82,12.20
2,1,9,2.588,91.3,15.1,0.82,12.01
3,1,12,1.334,27.4,5.4,0.32,4.91
4,1,15,1.076,17.0,3.5,0.19,3.27
...,...,...,...,...,...,...,...
181,33,13,1.693,23.0,7.8,0.18,3.45
182,34,1,1.294,18.4,6.3,0.12,2.18
183,34,4,1.302,17.0,6.3,0.12,2.16
184,34,7,1.305,17.3,6.3,0.13,2.17


## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [15]:
keep_param = ['bottle','prdm']
# keep_param = ['bottle','prsm']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(nut_data[nut_data['Cast']==matchcast],cdata.reset_index()[keep_param],right_on='bottle',left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
    except:
        continue

## Add Deployment meta information

In [16]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [17]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)

## Add institutional meta-information


In [18]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [20]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'Sil (uM)':'SI',
                            'PO4 (uM)':'PO4',
                            'NO2 (uM)':'NO2', 
                            'NO3 (uM)':'NO3',
                            'NH4 (uM)':'NH4',
                            'Sil Flag':'SI_WOCE_FLAG',
                            'PO4 Flag':'PO4_WOCE_FLAG',
                            'NO2 Flag':'NO2_WOCE_FLAG', 
                            'NO3 Flag':'NO3_WOCE_FLAG',
                            'NH4 Flag':'NH4_WOCE_FLAG',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")
    except:
        print(f'Skipping {cast}')

Skipping 012


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history